In [21]:
import jenkspy
import pandas as pd
from utils import PostGresLoad
from dissim_index import classify, categorize_dscore

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
pg = PostGresLoad()

In [23]:
df_11_12 = pg.df_from_rds("""select * from dissimilarity_index where "YEAR" = '2011-12';""")
df_13_14 = pg.df_from_rds("""select * from dissimilarity_index where "YEAR" = '2013-14';""")
df_15_16 = pg.df_from_rds("""select * from dissimilarity_index where "YEAR" = '2015-16';""")
df_17_18 = pg.df_from_rds("""select * from dissimilarity_index where "YEAR" = '2017-18';""")

In [24]:
frames = [df_11_12, df_13_14, df_15_16, df_17_18]

In [25]:
for df in frames:
    for col in df.columns:
        if 'd_' in col:
            df[f'classified_{col}'] = categorize_dscore(df[col])


/Users/cave/miniconda3/lib/python3.7/site-packages/jenkspy/core.py:64: UserWarning: Invalid values encountered (NaN or Inf) were ignored
  warnings.warn('Invalid values encountered (NaN or Inf) were ignored')


In [26]:
df_17_18.shape

(2934, 14)

In [27]:
print(df_15_16.shape)

(2455, 14)


In [28]:
print(df_13_14.shape)

(2448, 14)


In [31]:
df_final = pd.concat([df_11_12, df_13_14, df_15_16, df_17_18])
df_final.to_csv('/Users/cave/Desktop/pleiades_archive/discriminology/output/classified_scores.csv', index=False)

In [32]:
df_final.columns

Index(['LEAID', 'YEAR', 'd_wh_bl', 'd_wh_as', 'd_wh_am', 'd_wh_tr', 'd_wh_hp',
       'd_wh_hi', 'classified_d_wh_bl', 'classified_d_wh_as',
       'classified_d_wh_am', 'classified_d_wh_tr', 'classified_d_wh_hp',
       'classified_d_wh_hi'],
      dtype='object')